In [8]:
import random as rand
import numpy as np
import time
filenames = ['a_example','b_lovely_landscapes',
             'c_memorable_moments',
             'd_pet_pictures',
             'e_shiny_selfies']
exe = '.txt'
t0 = 1
t1 = 0
#max_iter = 50
def get_common_tags(image1, image2):
    return image1['tags'] & image2['tags']


def get_unique_tags(image1, image2):
    return image1['tags'] | image2['tags']


for filename in [filenames[1]]:
    print(filename + '-----------------------')
    filestart = time.time()
    # Input------------------
    with open(filename + exe,'r') as file:
        imagen = int(file.readline())
        images = list()
        for i,line in enumerate(file):
            tokens = line[:-1].split(' ')
            images.append({'id':i,'ori': tokens[0], 'tagn': int(tokens[1]), 'tags': set()})
            for token in tokens[2:]:
                images[i]['tags'].add(token)

    #Create slides--------------
    slides = []
    print('Creating slides')
    while len(images) > 0:
        if len(images) % 10000 == 0:
            print('images {} time {}'.format(len(images),time.time()-filestart))

        image = images.pop()
        if image['ori'] == 'H':
            # Create slide immedietly
            slides.append({'id':[image['id']],
                           'tags':image['tags'],
                           'tagn': image['tagn']})
        else:
            # Find partner for vertical image to create slide
            best_candidate = None
            index = None
            for i in range(len(images)):
                if images[i]['ori'] == 'H':
                    continue
                common = len(get_common_tags(image,images[i]))
                if common == 0:
                    best_candidate = (images[i],common)
                    index = i
                    break
                if best_candidate is None or common < best_candidate[1]:
                    best_candidate = (images[i],common)
                    index = i
            if best_candidate is None:
                continue
            last = images.pop()
            if index != len(images):
                images[index] = last
            combined_tags = get_unique_tags(image,best_candidate[0])
            slides.append({'id':[image['id'],best_candidate[0]['id']],
                           'tags': combined_tags,
                           'tagn':len(combined_tags)})
    # Sorting slides based on length
    print('Sorting slides')
    slides.sort(key = lambda slide: slide['tagn'])
    print('Ordering slides')
    # Ordering slides----------------------
    def get_score(slide1, slide2):
        diff = np.abs(len(slide1["tags"]) - len(slide2['tags']))
        ab = len(slide1["tags"] & slide2['tags'])
        anotb = len(slide1["tags"] - slide2['tags'])
        notab = len(slide2["tags"] - slide1['tags'])
        m = min(ab, anotb, notab)
        #print('diff:{}, ab: {}, anotb: {}, notab: {}, min{}'.format(diff,ab,anotb,notab,m))
        result = t0 * m - t1 * diff
        return result,m

    def get_perfect_score(slide):
        return t0 * int(slide['tagn'] / 2)

    ordered_slides = [slides.pop(0)]
    cummulative_score = 0
    while len(slides) > 0:
        if len(slides) % 100 == 0:
            print('images {}. score {}. time {}'.format(len(slides),cummulative_score,time.time()-filestart))
        perfectscore = get_perfect_score(ordered_slides[-1])
        best_candidate = (slides[0],get_score(slides[0],ordered_slides[-1]))
        index = 0
        for i,slide in enumerate(slides[1:]):
            s = get_score(slides[i], ordered_slides[-1])
            if s[0] > best_candidate[1][0]:
                best_candidate = (slides[i],s)
                index = i
            if s == perfectscore:
                break
        cummulative_score += best_candidate[1][1]
        ordered_slides.append(best_candidate[0])
        last = slides.pop()
        if index != len(slides):
            slides[index] = last

    # Output--------------------
    slides = ordered_slides
    with open(filename + '_out' + exe,'w') as f:
        f.write(str(len(slides))+ '\n')
        for slide in slides:
            if len(slide['id']) == 2:
                f.write(str(slide['id'][0]) + ' ' + str(slide['id'][1]) + '\n')
            else:
                f.write(str(slide['id'][0]) + '\n')
    print('Finished {} in {}'.format(filename,time.time()-filestart))



b_lovely_landscapes-----------------------
Creating slides
images 80000 time 0.6442763805389404
images 70000 time 0.6522550582885742
images 60000 time 0.7460043430328369
images 50000 time 0.752985954284668
images 40000 time 0.7609643936157227
images 30000 time 0.7689428329467773
images 20000 time 0.7759246826171875
images 10000 time 0.7829060554504395
Sorting slides
Ordering slides
images 79900. score 297. time 31.93910050392151
images 79800. score 597. time 61.88900303840637
images 79700. score 897. time 91.96756148338318
images 79600. score 1197. time 121.75888848304749
images 79500. score 1497. time 151.50932455062866
images 79400. score 1797. time 181.18894982337952
images 79300. score 2097. time 210.5743613243103
images 79200. score 2397. time 240.81847667694092
images 79100. score 2697. time 271.1382586956024
images 79000. score 2997. time 301.4222900867462
images 78900. score 3297. time 331.99360275268555
images 78800. score 3597. time 363.18284368515015
images 78700. score 3897

images 64300. score 47094. time 4537.8020758628845
images 64200. score 47391. time 4562.683940649033
images 64100. score 47691. time 4587.900822162628
images 64000. score 47991. time 4613.766870737076
images 63900. score 48291. time 4638.946088552475
images 63800. score 48591. time 4663.2859942913055
images 63700. score 48891. time 4688.214326381683
images 63600. score 49191. time 4713.205490589142
images 63500. score 49491. time 4738.535801649094
images 63400. score 49791. time 4763.0698664188385
images 63300. score 50091. time 4788.45881819725
images 63200. score 50391. time 4813.178171634674
images 63100. score 50691. time 4838.4235191345215
images 63000. score 50985. time 4862.865560054779
images 62900. score 51285. time 4887.739651441574
images 62800. score 51585. time 4913.098504543304
images 62700. score 51885. time 4937.918031454086
images 62600. score 52182. time 4962.328080654144
images 62500. score 52482. time 4987.6376712322235
images 62400. score 52782. time 5013.105535030

images 47900. score 96075. time 8195.191561937332
images 47800. score 96375. time 8214.151854515076
images 47700. score 96669. time 8232.946590662003
images 47600. score 96966. time 8251.65056848526
images 47500. score 97263. time 8270.431341409683
images 47400. score 97563. time 8289.879330158234
images 47300. score 97863. time 8308.468615531921
images 47200. score 98160. time 8327.4139482975
images 47100. score 98460. time 8345.806759119034
images 47000. score 98754. time 8364.634406805038
images 46900. score 99042. time 8383.25760102272
images 46800. score 99339. time 8402.158054351807
images 46700. score 99639. time 8421.27393078804
images 46600. score 99933. time 8439.970927715302
images 46500. score 100233. time 8458.41759443283
images 46400. score 100530. time 8477.385866641998
images 46300. score 100824. time 8495.712852478027
images 46200. score 101124. time 8513.86331152916
images 46100. score 101424. time 8532.535375118256
images 46000. score 101724. time 8550.651925086975
i

images 31800. score 143265. time 10771.142535448074
images 31700. score 143532. time 10783.239182949066
images 31600. score 143814. time 10795.820535421371
images 31500. score 144093. time 10808.256277561188
images 31400. score 144366. time 10820.428724050522
images 31300. score 144648. time 10832.51140999794
images 31200. score 144936. time 10844.892298936844
images 31100. score 145212. time 10857.101646184921
images 31000. score 145500. time 10869.052684545517
images 30900. score 145785. time 10881.14135503769
images 30800. score 146058. time 10892.953763961792
images 30700. score 146334. time 10904.960652828217
images 30600. score 146625. time 10916.723195552826
images 30500. score 146907. time 10928.599433898926
images 30400. score 147198. time 10940.57740020752
images 30300. score 147489. time 10952.631163835526
images 30200. score 147765. time 10965.558613061905
images 30100. score 148053. time 10977.680257797241
images 30000. score 148329. time 10989.483900308609
images 29900. s

images 15900. score 185829. time 12285.873163461685
images 15800. score 186063. time 12292.176306009293
images 15700. score 186318. time 12298.073534488678
images 15600. score 186558. time 12304.172224283218
images 15500. score 186801. time 12310.00163435936
images 15400. score 187050. time 12315.810100317001
images 15300. score 187296. time 12321.901808977127
images 15200. score 187524. time 12327.703293323517
images 15100. score 187770. time 12333.336228609085
images 15000. score 188016. time 12339.456859588623
images 14900. score 188235. time 12345.192521095276
images 14800. score 188481. time 12350.957103729248
images 14700. score 188712. time 12356.804465532303
images 14600. score 188946. time 12362.414462566376
images 14500. score 189189. time 12367.864885807037
images 14400. score 189420. time 12373.711250543594
images 14300. score 189651. time 12379.38208436966
images 14200. score 189885. time 12384.85345196724
images 14100. score 190113. time 12390.155272722244
images 14000. s